In [ ]:
from prepare_data import preprocess_data_czech
from field_info import FieldInfo
from tensor_encoder import TensorEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

: 

In [2]:
raw_data = pd.read_csv('../DATA/tr_by_acct_w_age.csv')
data, LOG_AMOUNT_SCALE, TD_SCALE, START_DATE, TCODE_TO_NUM, NUM_TO_TCODE = preprocess_data_czech(raw_data)
data2 = data[['account_id','age','tcode', 'tcode_num', 'datetime', 'month', 'dow', 'day', 'dtme' ,'log_amount_sc','td_sc']]
df= data2.copy()

n_tcodes = len(TCODE_TO_NUM)

info = FieldInfo(n_tcodes)

max_seq_len = 80
min_seq_len = 20
encoder = TensorEncoder(df, info, max_seq_len, min_seq_len)


In [3]:
from field_info import FieldInfo
ACTIVATIONS = {
    "td_sc": "relu",
    "log_amount_sc": "relu"
}
fieldInfo = FieldInfo(n_tcodes)
config = {}
config["ORDER"] = fieldInfo.DATA_KEY_ORDER
config["FIELD_STARTS_IN"] = fieldInfo.FIELD_STARTS_IN
config["FIELD_DIMS_IN"] = fieldInfo.FIELD_DIMS_IN
config["FIELD_STARTS_NET"] = fieldInfo.FIELD_STARTS_NET
config["FIELD_DIMS_NET"] = fieldInfo.FIELD_DIMS_NET
config["ACTIVATIONS"] = ACTIVATIONS

In [4]:
fieldInfo.FIELD_DIMS_IN

{'tcode_num': 16,
 'dow': 2,
 'month': 2,
 'day': 2,
 'dtme': 2,
 'td_sc': 1,
 'log_amount_sc': 1}

In [5]:
encoder.count_variable_length_seqs_with_overlap(), encoder.count_seqs_with_overlap(), encoder.count_seqs_in_df()

(99101, 72764, 14354)

In [6]:
encoder.encode()

2023-12-01 10:04:38.205908: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-01 10:04:38.207253: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-12-01 10:04:38.237707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-12-01 10:04:38.237880: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-12-01 10:04:38.237897: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-12-01 10:0

Finished encoding 2000 of 14354 seqs
Finished encoding 4000 of 14354 seqs
Finished encoding 6000 of 14354 seqs
Finished encoding 8000 of 14354 seqs
Finished encoding 10000 of 14354 seqs
Finished encoding 12000 of 14354 seqs
Finished encoding 14000 of 14354 seqs
Took 73.28 secs


In [7]:
encoder.inp_tensor.shape

(14354, 81, 26)

In [ ]:
encoder.inp_tensor

In [9]:
# Create a TensorFlow dataset from the entire tensors
ds_all = tf.data.Dataset.from_tensor_slices((encoder.inp_tensor.astype(np.float32), encoder.tar_tensor.astype(np.float32)))

def make_batches(ds, buffer_size, batch_size):
    return ds.cache().shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

BUFFER_SIZE = ds_all.cardinality().numpy()
bs = 64  # batch size

# Create train batches from the entire dataset
train_batches = make_batches(ds_all, BUFFER_SIZE, bs)

In [10]:
train_batches

<PrefetchDataset shapes: ((None, 81, 26), (None, 80, 7)), types: (tf.float32, tf.float32)>

In [11]:
# create only one sample of inp and tar to trace one iteration of loop
j = 0
for (batch_no, (inp, tar)) in enumerate(train_batches):
    if j == 0:
       break

In [ ]:
inp

In [12]:
from modules import InputEmbedLayer,InputEmbedLayer_Res, ResidualLayer, RandomNoise_Simulator_Normal, positional_encoding, MultiHeadAttention, Transformer, create_masks, DecoderLayer, Decoder
import tensorflow as tf
features = 26
d_embedding = 128
dff = 128
d_model = 128
batch_size = 64
seq_len = 80
maximum_position_encoding = 256
rate = 0.1
num_heads = 2
num_layers = 4
deepmodel = Transformer(features, dff, d_embedding, d_model, maximum_position_encoding,num_heads, num_layers,config, rate=0.1)

In [13]:
transformer  = deepmodel.make_transformer(tar, inp)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [ ]:
transformer(inp[:, :-1])

In [ ]:
transformer.summary()

In [ ]:
loss_scce_logit = SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [23]:
LOSS_TYPES = fieldInfo.LOSS_TYPES
LOSS_TYPES

{'day': 'scce',
 'dtme': 'scce',
 'dow': 'scce',
 'month': 'scce',
 'tcode_num': 'scce',
 'td_sc': 'pdf',
 'log_amount_sc': 'pdf'}

In [14]:
from modules import InputEmbedLayer,InputEmbedLayer_Res, ResidualLayer, RandomNoise_Simulator_Normal, positional_encoding, MultiHeadAttention, create_masks, DecoderLayer, Decoder
import tensorflow as tf
features = 26
d_embedding = 128
dff = 128
d_model = 128
batch_size = 64
seq_len = 80
maximum_position_encoding = 256
rate = 0.1
num_heads = 2
num_layers = 4
z = RandomNoise_Simulator_Normal(batch_size, seq_len, features)

#Transformer Model
input_ = tf.keras.layers.Input(shape=(None, features))
x = InputEmbedLayer(features, dff , d_embedding)(input_)

pos_encoding = positional_encoding(maximum_position_encoding, d_embedding)   #(1, maximum_position_encoding=256, d_model=128)

seq_len = tf.shape(x)[1]
x += pos_encoding[:, :seq_len, :]     #x is the output of Input layer

x = tf.keras.layers.Dropout(rate)(x, training=True)

mask, _ = create_masks(tar)
d_inp_decoder = tf.keras.backend.int_shape(x)[-1]

out, attention_weights = Decoder(num_layers, d_inp_decoder, d_model, num_heads, dff)(x, True, mask)

final_output = tf.keras.layers.Dense(d_model, activation=None)(out)
model = tf.keras.models.Model(input_, final_output)


In [ ]:
model.summary()

In [18]:
# from modules import InputEmbedLayer,InputEmbedLayer_Res, ResidualLayer, RandomNoise_Simulator_Normal, positional_encoding, MultiHeadAttention, create_masks, DecoderLayer
# import tensorflow as tf
# features = 26
# d_embedding = 128
# dff = 128
# d_model = 128
# batch_size = 64
# seq_len = 80
# maximum_position_encoding = 256
# rate = 0.1
# num_heads = 2
# num_layers = 4
# z = RandomNoise_Simulator_Normal(batch_size, seq_len, features)

# #Transformer Model
# input_ = tf.keras.layers.Input(shape=(None, features))
# x = InputEmbedLayer(features, dff , d_embedding)(input_)

# pos_encoding = positional_encoding(maximum_position_encoding, d_embedding)   #(1, maximum_position_encoding=256, d_model=128)

# seq_len = tf.shape(x)[1]
# x += pos_encoding[:, :seq_len, :]     #x is the output of Input layer

# x = tf.keras.layers.Dropout(rate)(x, training=True)

# attention_weights = {}
# mask, _ = create_masks(tar)
# for i in range(num_layers):
#     d_inp_decoder = tf.keras.backend.int_shape(x)[-1]
#     x, attentionweights = DecoderLayer(d_inp_decoder, d_model, num_heads, dff)(x, True, mask)
#     attention_weights['decoder_layer{}'.format(i+1)] = attentionweights

# final_output = tf.keras.layers.Dense(d_model, activation=None)(x)


# model = tf.keras.models.Model(input_, final_output)


In [ ]:
model(inp[:, :-1])

In [17]:
attention_weights

{'decoder_layer1': <KerasTensor: shape=(64, 2, 80, 80) dtype=float32 (created by layer 'decoder_3')>,
 'decoder_layer2': <KerasTensor: shape=(64, 2, 80, 80) dtype=float32 (created by layer 'decoder_3')>,
 'decoder_layer3': <KerasTensor: shape=(64, 2, 80, 80) dtype=float32 (created by layer 'decoder_3')>,
 'decoder_layer4': <KerasTensor: shape=(64, 2, 80, 80) dtype=float32 (created by layer 'decoder_3')>}

In [ ]:
config = {}
FIELD_DIMS_IN, FIELD_STARTS_IN, FIELD_DIMS_TAR, FIELD_STARTS_TAR, FIELD_DIMS_NET, FIELD_STARTS_NET = get_field_info(ds_suffix)

config["ORDER"] = DATA_KEY_ORDER
config["FIELD_STARTS_IN"] = FIELD_STARTS_IN
config["FIELD_DIMS_IN"] = FIELD_DIMS_IN
config["FIELD_STARTS_NET"] = FIELD_STARTS_NET
config["FIELD_DIMS_NET"] = FIELD_DIMS_NET


config["ACTIVATIONS"] = ACTIVATIONS